In [2]:
# %load /home/jonathan/.ipython/profile_default/startup/startup-01.py
# start up settings for ipython
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys

plt.style.use('ggplot')
plt.rcParams['font.size'] = 15.0
plt.rcParams['axes.labelsize'] = 15.0
plt.rcParams['xtick.labelsize'] = 15.0
plt.rcParams['ytick.labelsize'] = 15.0
plt.rcParams['legend.fontsize'] = 15.0

%matplotlib inline

# set the max column width
pd.options.display.max_colwidth = 1000


In [55]:
tender_df = pd.read_csv('../../data/processed14Nov/tender.csv')
tprod_df = pd.read_csv('../../data/processed14Nov/tenderProduct.csv')

In [56]:
tender_df.describe()

,AdjudicacionNumeroOferentes,AdjudicacionTipo,CantidadReclamos,CodigoEstado,CodigoTipo,CompradorCodigoOrganismo,CompradorCodigoUnidad,CompradorCodigoUsuario,Contrato,DiasCierreLicitacion,...,TipoConvocatoria,TipoPago,TomaRazon,UnidadTiempo,UnidadTiempoContratoLicitacion,UnidadTiempoDuracionContrato,UnidadTiempoEvaluacion,ValorTiempoRenovacion,VisibilidadMonto,tender_product_duplicated
count,16593.000000,13440.000000,17165.000000,17165.000000,17165.000000,1.716500e+04,17165.000000,1.716500e+04,16995.000000,17165.0,...,17165.000000,17165.000000,17165.000000,17165.000000,17165.000000,17165.000000,16623.000000,17165.0,17165.000000,9.000000
mean,4.590671,2.787128,311.784212,7.869152,1.025750,8.036910e+04,54763.002738,1.008100e+06,0.318800,0.0,...,0.987125,2.664200,0.003495,1.047713,0.750539,1.490883,1.049269,0.0,0.370987,14.333333
std,5.701965,1.144841,530.086641,0.890213,0.225477,1.367779e+05,181734.125820,4.825238e+05,0.607742,0.0,...,0.112739,1.147034,0.059021,0.239912,0.699144,1.059570,0.243635,0.0,0.483083,12.825755
min,0.000000,1.000000,0.000000,6.000000,1.000000,6.918000e+03,1005.000000,5.000000e+00,0.000000,0.0,...,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.0,0.000000,3.000000
25%,1.000000,2.000000,43.000000,8.000000,1.000000,7.319000e+03,3002.000000,5.263950e+05,0.000000,0.0,...,1.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.0,0.000000,6.000000
50%,3.000000,2.000000,122.000000,8.000000,1.000000,8.687200e+04,3726.000000,1.217042e+06,0.000000,0.0,...,1.000000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.000000,10.000000
75%,6.000000,4.000000,309.000000,8.000000,1.000000,1.138120e+05,4947.000000,1.374063e+06,0.000000,0.0,...,1.000000,4.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,20.000000
max,69.000000,6.000000,3191.000000,16.000000,3.000000,1.512218e+06,931768.000000,1.529323e+06,2.000000,0.0,...,1.000000,4.000000,1.000000,4.000000,2.000000,4.000000,4.000000,0.0,1.000000,44.000000


In [57]:
describe_df = tender_df.describe()
for colname in describe_df.columns:
    if describe_df.loc['std', colname] == 0:
        print(colname)

DiasCierreLicitacion
EsBaseTipo
EsRenovable
ValorTiempoRenovacion


In [58]:
tsmall_df = tender_df.drop(['DiasCierreLicitacion', 'EsBaseTipo', 'EsRenovable', 'ValorTiempoRenovacion'], axis='columns')

In [59]:
tender_df.shape

(17165, 84)

In [60]:
tsmall_df['DireccionEntrega'].unique()

array([ nan])

In [61]:
perc_null_series = tsmall_df.isnull().sum() / 17165

all_null_series = perc_null_series[(perc_null_series == 1.0)]

# tsmall_df = tsmall_df.drop(all_null_series.index.values, axis='columns')

In [62]:
very_null_series = perc_null_series[perc_null_series > 0.9]
very_null_series_cols = very_null_series.index.values
very_null_series_cols = [colname for colname in very_null_series_cols if 'tender_product_duplicated' not in colname]

In [63]:
very_null_series_cols

['DireccionEntrega',
 'DireccionVisita',
 'FechaCierre',
 'FechasFechaEntregaAntecedentes',
 'FechasFechaEstimadaFirma',
 'FechasFechaSoporteFisico',
 'FechasFechaTiempoEvaluacion',
 'FechasFechaVisitaTerreno',
 'JustificacionPublicidad',
 'ObservacionContract',
 'PeriodoTiempoRenovacion',
 'TipoDuracionContrato']

In [45]:
tsmall_df = tsmall_df.drop(very_null_series_cols, axis='columns')

### Remove cases which has 1 tender product

In [46]:
one_tprod_df = tender_df[(tender_df.ItemsCantidad == 1)]
tsmall_df = tsmall_df[~(tsmall_df['CodigoExterno'].isin(one_tprod_df['CodigoExterno'].values))]

In [47]:
outdir = os.path.join('..', '..', 'data', 'processed18Nov')
tsmall_df.to_csv(os.path.join(outdir, 'tsmall.csv'), index=None)

In [64]:
describe_df = tprod_df.describe()
for colname in describe_df.columns:
    if describe_df.loc['std', colname] == 0:
        print(colname)

In [65]:
perc_null_series = tprod_df.isnull().sum() / tprod_df.shape[0]

all_null_series = perc_null_series[(perc_null_series == 1.0)]

# tsmall_df = tsmall_df.drop(all_null_series.index.values, axis='columns')

In [66]:
very_null_series = perc_null_series[perc_null_series > 0.8]
very_null_series_cols = very_null_series.index.values

### Remove tender products which are of 1 tender product

In [51]:
tsprod_df = tprod_df[~(tprod_df['CodigoExterno'].isin(one_tprod_df['CodigoExterno'].values))]

In [52]:
tsprod_df.to_csv(os.path.join(outdir, 'tsprod.csv'), index=None)